In [ ]:
!pip install --upgrade pip
!pip install faiss-cpu sentence-transformers transformers accelerate

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

In [ ]:
import json
import faiss
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# ✅ OpenChat 모델 로딩
model_id = "openchat/openchat-3.5-0106"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

chat = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=768,
    temperature=0.7,
    do_sample=True
)

# ✅ game.json 로딩
with open("game.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [item["text"] for item in data]
game_names = [item["game_name"] for item in data]

# ✅ 임베딩 모델 로딩
embed_model = SentenceTransformer("BAAI/bge-m3", device="cuda")
embeddings = embed_model.encode(
    texts,
    normalize_embeddings=True,
    batch_size=8,
    show_progress_bar=True
)

# ✅ FAISS 인덱스 구축
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(np.array(embeddings))

# ✅ 출력 후처리 함수
def clean_output(raw_output: str) -> str:
    # assistant 토큰 이후만 추출
    if "<|assistant|>" in raw_output:
        raw_output = raw_output.split("<|assistant|>")[-1]

    # "추천 완료!" 이전까지만 유지
    if "추천 완료!" in raw_output:
        raw_output = raw_output.split("추천 완료!")[0]

    return raw_output.strip()

# ✅ 추천 함수
def recommend_with_openchat(query, default_k=5):
    number_match = re.search(r'(\d+)\s*개', query)
    top_k = int(number_match.group(1)) if number_match else default_k

    # 유사도 검색
    query_embedding = embed_model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(query_embedding), k=top_k)

    # 검색 결과 → 프롬프트 구성용
    retrieved = []
    selected_names = []
    for i in I[0]:
        name = game_names[i]
        desc = texts[i]
        retrieved.append(f"[{name}]\n{desc}")
        selected_names.append(f"- {name}")

    context = "\n\n".join(retrieved)
    name_list_str = "\n".join(selected_names)

    # 프롬프트
    prompt = f"""아래는 보드게임 설명입니다. 각 게임은 "[게임명]\n설명" 형식으로 되어 있습니다.

[게임 설명]
{context}

⚠️ 반드시 아래의 게임 이름 목록 중에서만 선택할 수 있습니다. 이 목록에 없는 게임 이름을 절대 생성하지 마세요.
만약 목록에 없는 게임명을 출력하면 실패로 간주됩니다.

[허용된 게임 이름 목록]
{ name_list_str }

[사용자 질문]
{query}

📌 출력 지침:
- 반드시 위 목록에 있는 게임 중에서만 5개를 골라 추천하세요.
- 출력 형식은 반드시 아래 형식처럼 작성하세요:

게임명1: 추천 이유
게임명2: 추천 이유
게임명3: 추천 이유
게임명4: 추천 이유
게임명5: 추천 이유

- 각 줄은 '게임명: 추천 이유' 형식으로만 작성하고, 줄바꿈 이외에 아무 포맷도 쓰지 마세요.
- 추천이 모두 끝나면 마지막 줄에 반드시 다음과 같이 써주세요:
추천 완료!

그 이후에는 아무 것도 쓰지 마세요.
"""

    chat_prompt = f"<|system|>\n너는 보드게임 추천 도우미야. 사용자의 질문에 따라 관련 게임을 추천하고 이유도 알려줘.\n<|user|>\n{prompt}\n<|assistant|>"
    raw_output = chat(chat_prompt)[0]["generated_text"]
    cleaned = clean_output(raw_output)

    print("\n🤖 추천게임 (OpenChat 기반):\n")
    print(cleaned)

In [ ]:
# ✅ 실행
if __name__ == "__main__":
    print("🎲 보드게임 추천 시스템 (OpenChat 기반)")
    user_query = input("❓ 어떤 게임을 원하시나요?\n예시: '전략적이고 빠른 게임 3개 추천해줘'\n> ")
    recommend_with_openchat(user_query)